In [ ]:
"""
# **PoSh Multimodal Forgery Detection System**

# **Mount Drive**
"""

from google.colab import drive
drive.mount('/content/drive')

!pip install audio-extract
!pip install moviepy
!pip install facenet_pytorch #change made
!pip install timm

%cd /content/drive/Shareddrives/FYP/Swin-Transformer

# Imports Specific to Swin Transformer
from swin_functions_and_classes import *
from going_modular.going_modular import engine
from helper_functions import set_seeds

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 27.5 MB/s eta 0:00:00
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.4.9
    Uninstalling imageio-ffmpeg-0.4.9:
      Successfully uninstalled imageio-ffmpeg-0.4.9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand

In [ ]:
def multimodal_forgery_detection(Video):

    """# **Imports**"""

    import os
    import shutil
    from google.colab import files
    from IPython.display import HTML, clear_output
    from base64 import b64encode
    import moviepy.editor as mp

    from audio_extract import extract_audio
    from moviepy.editor import VideoFileClip
    import librosa.display
    import numpy as np
    import matplotlib.pyplot as plt
    from google.colab.patches import cv2_imshow

    import PIL
    from PIL import Image
    import torch
    import torch.nn as nn
    from timm.models.layers import DropPath, to_2tuple, trunc_normal_
    import torchvision.transforms as transforms
    from torchvision import datasets, transforms
    from torch.utils.data import DataLoader, TensorDataset

    import cv2

    import warnings
    import pandas as pd
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers

    import torchvision.models as models
    import dlib

    from IPython.display import HTML, Audio, clear_output
    from google.colab.output import eval_js
    from base64 import b64decode
    from scipy.io.wavfile import read as wav_read
    import io
    import ffmpeg

    """**Upload Video**"""

    if not os.path.isfile(Video):
        print("ERROR: File not found!")
        raise SystemExit(0)
    else:
        shutil.copy(Video, '/content/input_video.mp4')

    """### **Split input video into audio, video and extract mel spectrogram**

    **Extract Mel Spectrogram**
    """

    def extract_mel_spectrogram(output_audio_path, output_image_path):
        y, sr = librosa.load(output_audio_path)
        # Compute the Mel spectrogram
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=sr//2)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        # Plot the Mel spectrogram
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(mel_spec_db, x_axis='time', y_axis='mel', sr=sr, fmax=sr//2)
        plt.colorbar(format='%+2.0f dB')
        plt.title('Mel spectrogram')
        plt.tight_layout()
        # Save the image
        plt.savefig(output_image_path)

    """**Save Audio and Melspectrogram**"""

    # Define the video file path
    video_path = "/content/input_video.mp4"

    # Load the video clip
    video_clip = VideoFileClip(video_path)

    # Extract the audio from the video
    audio_clip = video_clip.audio

    # Define the audio file path
    audio_path = "/content/audio.mp3"

    # Write the extracted audio to a file
    audio_clip.write_audiofile(audio_path, codec='mp3')  # You can use other audio codecs like 'wav' if needed

    # Close the audio and video clips
    audio_clip.close()
    video_clip.close()

    # Define the melspectrogram path
    melspectrogram_path = "/content/melspectrogram.png"
    extract_mel_spectrogram(audio_path, melspectrogram_path)

    print("Video saved to:", video_path)
    print("Audio Saved to:", audio_path)
    print("Melspectrogram Saved to:", melspectrogram_path)

    """# **Audio Network**"""

    """**Prediction**"""

    # import torchvision.transforms as transforms

    # Create image size
    IMG_SIZE = 224

    # Create transform pipeline manually
    manual_transforms = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
    ])

    # Define the class names
    class_names = ['fake', 'real']

    model = SwinTransformer(num_classes=len(class_names))

    # Load saved model
    model.load_state_dict(torch.load('/content/drive/Shareddrives/FYP/trained_model_SWIN_8_1e-4_0.01_5epochs.pth'))
    model.eval()

    # Open melspectrogram
    image = Image.open('/content/melspectrogram.png')
    image_rgb = image.convert("RGB")

    # Apply the transformation to convert the image to a tensor
    image_tensor = manual_transforms(image_rgb).unsqueeze(0)  # Add batch dimension

    # Perform inference
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
        audio_network_prediction = class_names[predicted[0]]

    """#**Video Network**"""

    %run /content/drive/Shareddrives/FYP2/Video_Swin_Transformer/predict_folder.py --weights-dir /content/drive/Shareddrives/FYP2/Video_Swin_Transformer/weights --models video_swin_weights --test-dir /content/
    %store video_network_prediction

    """# **Audio-Visual Network**

    ### **Wav2lip**

    **Setup**
    """

    %cd /content/

    !rm -rf /content/sample_data
    !mkdir /content/sample_data

    !git clone https://github.com/justinjohn0306/Wav2Lip

    %cd /content/Wav2Lip

    #download the pretrained model
    !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
    !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
    !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
    !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'
    !pip install git+https://github.com/elliottzheng/batch-face.git@master

    !pip install ffmpeg-python mediapipe==0.8.11

    %cd content

    def showVideo(path):
      mp4 = open(str(path),'rb').read()
      data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
      return HTML("""
      <video width=700 controls>
            <source src="%s" type="video/mp4">
      </video>
      """ % data_url)

    clear_output()

    """**Lip Sync**"""

    %cd /content/Wav2Lip

    # Set up paths and variables for the output file
    output_file_path = '/content/Wav2Lip/results/result_voice.mp4'

    # Delete existing output file before processing, if any
    if os.path.exists(output_file_path):
        os.remove(output_file_path)

    pad_top =  0
    pad_bottom =  10
    pad_left =  0
    pad_right =  0
    rescaleFactor =  1
    nosmooth = True
    use_hd_model = True

    checkpoint_path = 'checkpoints/wav2lip.pth' if not use_hd_model else 'checkpoints/wav2lip_gan.pth'


    if nosmooth == False:
      !python inference.py --checkpoint_path $checkpoint_path --face "/content/input_video.mp4" --audio "/content/audio.mp3" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor
    else:
      !python inference.py --checkpoint_path $checkpoint_path --face "/content/input_video.mp4" --audio "/content/audio.mp3" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth

    #Preview output video
    if os.path.exists(output_file_path):
        clear_output()
        print("Final Video Preview")
        print("Download this video from", output_file_path)
        showVideo(output_file_path)
    else:
        print("Processing failed. Output video not found.")

    shutil.copy('/content/Wav2Lip/results/result_voice.mp4', '/content/wav2lip.mp4')

    """### **Real-ESRGAN**

    **Setup**
    """

    # Clone Real-ESRGAN and enter the Real-ESRGAN
    %cd /content/
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    %cd /content/Real-ESRGAN
    # Set up the environment
    !pip install basicsr
    !pip install facexlib
    !pip install gfpgan
    !pip install ffmpeg-python
    !pip install -r requirements.txt
    !python setup.py develop

    """**Fix basicsr Error**"""

    def read_file_to_list(input_file_path):
        with open(input_file_path, 'r') as file:
            lines = file.readlines()
        return lines

    def write_list_to_file(lines, output_file_path):
        with open(output_file_path, 'w') as file:
            file.writelines(lines)

    # Example usage
    input_file_path = '/usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py'  # Replace with your input file path
    output_file_path = '/usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py'  # Replace with your output file path

    # Read contents of input file into a list
    lines = read_file_to_list(input_file_path)
    lines[7] = "from torchvision.transforms.functional import rgb_to_grayscale\n"

    # Write the list into another file
    write_list_to_file(lines, output_file_path)

    """**Create Folders for Videos**"""

    upload_folder = '/content/Real-ESRGAN/upload'
    result_folder = '/content/Real-ESRGAN/results'

    if os.path.isdir(upload_folder):
        shutil.rmtree(upload_folder)
    if os.path.isdir(result_folder):
        shutil.rmtree(result_folder)
    os.mkdir(upload_folder)
    os.mkdir(result_folder)

    shutil.copy('/content/input_video.mp4', upload_folder + "/input_video.mp4")
    shutil.copy('/content/wav2lip.mp4', upload_folder + "/wav2lip.mp4")

    """**Improve Resolution**"""

    ! python inference_realesrgan_video.py -i upload/input_video.mp4 -n realesr-animevideov3 -s 2 #--suffix out
    # Arguments
    # -i, --input: input video
    # -n, --model_name: Used model name
    # -s, --outscale: Scale
    # -suffix: Suffix of the output video

    ! python inference_realesrgan_video.py -i upload/wav2lip.mp4 -n realesr-animevideov3 -s 2

    """**Move Videos to '/content/'**"""

    shutil.copy(result_folder + '/input_video_out.mp4', "/content/input_video_out.mp4")
    shutil.copy(result_folder + '/wav2lip_out.mp4', "/content/wav2lip_out.mp4")

    """### **ResNet18**

    **Extract Embeddings**
    """

    import torch
    import torchvision.transforms as transforms
    import torchvision.models as models
    import cv2
    import dlib
    from PIL import Image
    # Load the pre-trained face detector from dlib
    detector = dlib.get_frontal_face_detector()

    # Preprocess the video frames
    def preprocess_frame(frame):
        transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize to match ResNet input size
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
        ])
        return transform(frame)

    # Load ResNet model
    resnet = models.resnet18(pretrained=True)
    resnet.fc = nn.Identity()  # Remove the fully connected layer

    # Extract features from video
    def extract_features(video_path):
        video_capture = cv2.VideoCapture(video_path)
        features = []
        while True:
            ret, frame = video_capture.read()
            if not ret:
                break

            # Detect faces in the frame
            faces = detector(frame)

            for face in faces:
                # Get the coordinates of the face
                x, y, w, h = face.left(), face.top(), face.width(), face.height()

                # Define region of interest (ROI) for lips detection
                lips_roi = frame[y + h//2 : y + h, x : x + w]
                # cv2_imshow(lips_roi)

            frame = Image.fromarray(cv2.cvtColor(lips_roi, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
            preprocessed_frame = preprocess_frame(frame)


            with torch.no_grad():
                features.append(resnet(preprocessed_frame.unsqueeze(0)))
        video_capture.release()
        return torch.stack(features)

    # Aggregate features
    def aggregate_features(features):
        # Simple aggregation by averaging the features
        return torch.mean(features, dim=0)


    video_path = '/content/input_video_out.mp4'
    video_features = extract_features(video_path)
    video_embedding = aggregate_features(video_features)
    # Expected output: torch.Size([512])

    syn_video_path = '/content/wav2lip_out.mp4'
    syn_video_features = extract_features(syn_video_path)
    syn_video_embedding = aggregate_features(syn_video_features)
    # Expected output: torch.Size([512])

    """### **MS-TCN**

    **Setup**
    """

    %cd /content/
    !git clone https://github.com/ahaliassos/LipForensics.git
    %cd /content/LipForensics/models

    from tcn import MultibranchTemporalConvNet

    """**Testing**"""

    import torch
    from torch.utils.data import DataLoader, TensorDataset
    from tcn import MultibranchTemporalConvNet

    # Assuming test_embeddings is a list of tensors
    test_embeddings = [video_embedding - syn_video_embedding]

    # Convert data to PyTorch tensors
    test_embeddings_tensor = torch.stack(test_embeddings)
    test_embeddings_tensor = test_embeddings_tensor.permute(0, 2, 1)

    # Create TensorDataset
    test_dataset = TensorDataset(test_embeddings_tensor)

    # Create DataLoader
    batch_size = 32
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Load the pre-trained model
    model = MultibranchTemporalConvNet(num_inputs=512, num_channels=[60, 120, 240, 480], tcn_options={"kernel_size": [3, 5, 7]})
    model.load_state_dict(torch.load("/content/drive/Shareddrives/FYP/tcn_model_new_ss_196.pth"))
    model.eval()

    # Apply threshold for classification
    threshold = 0.5  # Adjust this threshold based on your needs

    # Testing loop
    predicted_labels = []
    with torch.no_grad():
        for data in test_loader:
            data = data[0]  # Extract the tensor from the list
            outputs = model(data)
            probabilities = torch.sigmoid(outputs)  # Apply sigmoid activation to convert logits to probabilities
            labels = (probabilities > threshold).float()  # Apply threshold for classification
            predicted_labels.append(labels)

    # Concatenate predicted labels
    predicted_labels = torch.cat(predicted_labels, dim=0)

    num_fake_frames = torch.sum(predicted_labels > threshold).item()
    total_frames = predicted_labels[0].size(0)
    fake_percentage = num_fake_frames / total_frames

    if fake_percentage > 0.5:
        audio_visual_network_prediction = 'fake'
    else:
        audio_visual_network_prediction = 'real'

    """# **OR Module**"""

    clear_output()

    forgery = ""
    if audio_network_prediction == 'fake' or video_network_prediction == 'fake' or audio_visual_network_prediction == 'fake':
        forgery += "Forged"
    else:
        forgery += "Not Forged"

    return (audio_network_prediction,
            video_network_prediction,
            audio_visual_network_prediction,
            forgery,
            '/content/melspectrogram.png',
            '/content/input_video_out.mp4',
            '/content/wav2lip_out.mp4')

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.5 MB/s

In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=multimodal_forgery_detection,
    inputs=[gr.Video(label="Input Video")],
    outputs=[gr.Textbox(label="Audio Network Prediction"),
             gr.Textbox(label="Video Network Prediction"),
             gr.Textbox(label="Audio-Visual Network Prediction"),
             gr.Textbox(label="Multimodal Forgery Detection"),
             gr.Image(label="Mel Spectrogram"),
             gr.Video(label="Input Video with Improved Resolution"),
             gr.Video(label="Wav2lip-Generated Video with Improved Resolution")],
    title="PoSh Multimodal Forgery Detection System",
    description="",
    allow_flagging=False,
    examples=[["/content/drive/Shareddrives/FYP/dataset/video/fake_video-fake_audio/Asian (East)/men/fvfa_Asian_East_men_0.mp4"],
              ["/content/drive/Shareddrives/FYP/dataset/video/real_video-real_audio/Caucasian (American)/women/rvra_Caucasian_American_women_14.mp4"],
              ["/content/drive/Shareddrives/FYP/testcases/testcase.mp4"],
              ["/content/drive/Shareddrives/FYP/testcases/testcase1.mp4"],
              ["/content/drive/Shareddrives/FYP/testcases/testcase2.mp4"],
              ["/content/drive/Shareddrives/FYP/mstcn_videos/fvra_Caucasian_European_men_0.mp4"],
              ["/content/drive/Shareddrives/FYP/dataset/video/real_video-fake_audio/African/women/rvfa_African_women_43.mp4"],
              ["/content/drive/Shareddrives/FYP/dataset/mstcn/fake/nonsync.mp4"]],
    theme='gradio/soft'
)
iface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:374: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7c2e495ef3137ebe97.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
